In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Model, Input
from keras.layers import Embedding, Flatten, dot
%matplotlib inline

dataset source: https://www.kaggle.com/prajitdatta/movielens-100k-dataset

In [2]:
path = '../../data/kaggle_movielens_100k/'

In [11]:
def read_rating_data(filename):
    return pd.read_csv(path+filename, sep='\t', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])

In [16]:
train_df = read_rating_data('ua.base')
test_df = read_rating_data('ua.test')
full_df = pd.concat([train_df, test_df])

In [17]:
pd.crosstab(full_df['userId'], full_df['movieId'], full_df['rating'], aggfunc=np.sum)

movieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
n_users = full_df['userId'].nunique()
n_movies = full_df['movieId'].nunique()
n_factors = 128

## Dot Product

In [36]:
u_in = Input(shape=(1, ), dtype=np.int64, name='user_in')
m_in = Input(shape=(1, ), dtype=np.int64, name='movie_in')

u_emb = Embedding(n_users, n_factors)(u_in)
m_emb = Embedding(n_movies, n_factors)(m_in)

x = dot([u_emb, m_emb])
# x = Fal

model = Model([u_in, m_in], x)
model.summary()

TypeError: dot() missing 1 required positional argument: 'axes'